# Setting up

In [ ]:
! pip install umap-learn
! pip install pandas


In [ ]:
# Define the current path
from pathlib import Path
root = Path.cwd() / 'data'


In [ ]:
# define the local path
vec_path = root / '3.0vec' / 'fast_vec.tsv'
metlab_path = root / '3.0vec' / 'fast_met_lab.tsv'
save_path = root / '5.0minivec'


# Load data

load vector data

In [ ]:
import pandas as pd


In [ ]:
df_vec = pd.read_csv(vec_path, sep='\t', header=None)
df_metlab = pd.read_csv(metlab_path, sep='\t')


# Define func

In [ ]:
from sklearn.manifold import TSNE


def tsne(v):
    """get df and return 2or3 D data based on TSNE"""
    tsne = TSNE(n_components=2,  # Dimension of the embedded space
                perplexity=600,
                learning_rate='auto',
                n_iter=5000,  # Maximum number of iterations
                init='pca'  # Initialization of embedding
                )

    df_tsne = tsne.fit_transform(v)  # execute tsne
    df_tsne = pd.DataFrame(df_tsne, columns=['x', 'y'])  # Rename labels

    # Combine tsne-d data with meta & label
    global df_tsne_metlab
    df_tsne_metlab = df_metlab.join(df_tsne)
    df_tsne_metlab.name = 'df_tsne_metlab'

    # Delete words with label 'general'
    global df_tsne_drop
    df_tsne_drop = df_tsne_metlab.set_index(
        ['word', 'discipline']).drop('general', level='discipline')
    df_tsne_drop = df_tsne_drop.reset_index()
    df_tsne_drop.name = 'df_tsne_drop'


In [ ]:
import umap


def umap_go(v):
    mapper = umap.UMAP(random_state=0,
                       n_neighbors=10,
                       min_dist=0.0,
                       n_components=2,
                       )

    df_umap = mapper.fit_transform(v)
    df_tsne = pd.DataFrame(df_umap, columns=['x', 'y'])  # Rename labels

    # Combine tsne-d data with meta & label
    global df_umap_metlab
    df_umap_metlab = df_metlab.join(df_tsne)
    df_umap_metlab.name = 'df_umap_metlab'

    # Delete words with label 'general'
    global df_umap_drop
    df_umap_drop = df_umap_metlab.set_index(
        ['word', 'discipline']).drop('general', level='discipline')
    df_umap_drop = df_umap_drop.reset_index()
    df_umap_drop.name = 'df_umap_drop'


In [ ]:
def tsv(df):
    path = df.name + '.tsv'
    with open(save_path / path, 'w', newline='\n') as s:
        df.to_csv(s,
                  sep='\t',
                  index=False)


# Execute

T-SNE

In [ ]:
tsne(df_vec)
tsv(df_tsne_metlab)
tsv(df_tsne_drop)


UMAP

In [ ]:
umap_go(df_vec)
tsv(df_umap_metlab)
tsv(df_umap_drop)
